In [73]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [74]:
class convolution():
    def __init__(self, in_channels, out_channels, kernel_size, padding = [0,0], stride = [1,1]):
        #self.width_in, self.height_in, _ = image.shape
        self.size_padding = padding
        self.size_stride_x = stride[0]
        self.size_stride_y = stride[1]
        self.width_filter = kernel_size[0]
        self.height_filter = kernel_size[1]
        self.num_in_channels = in_channels
        self.num_out_channels = out_channels
        
        #self.width_channel = (self.width_in - self.width_filter + 2*size_padding)/size_stride + 1
        #self.height_channel = (self.height_in - self.height_filter + 2*size_padding)/size_stride + 1
        
        self.filters = [[[[random.random() for width in range(self.width_filter)] for height in range(self.height_filter)] 
                        for deep in range(self.num_in_channels)] for num_filter in range(self.num_out_channels)]
        self.bias = [1 for i in range(self.num_out_channels)]
    def forward(self, in_image):
        #print(len(in_image[0]), self.width_filter, self.size_padding[0], self.size_stride_x)
        kanals, y, x = np.array(in_image).shape
        x_layer = int((x - self.width_filter + 2*self.size_padding[0])/self.size_stride_x + 1)
        y_layer = int((y - self.height_filter + 2*self.size_padding[0])/self.size_stride_y + 1)
        for i in range(kanals):
            
            in_image[i] = self.padding(in_image[i])
            
        kanals, y, x = np.array(in_image).shape
        #y = in_image.shape[1] - self.height_filter + 1
        #x = in_image.shape[0] - self.width_filter + 1
        out_filter = np.zeros((x_layer,y_layer), float)
        #out_channel = np.zeros(x,y,self.num_in_channels)
        #out_image = np.zeros(x,y,self.num_in_channels, self.num_out_channels)
        out_image = []
        for num_filter in range(self.num_out_channels):
            out_channel = []
            for deep_filter in range(self.num_in_channels):
                x_step = 0
                y_step = 0
                image = in_image[deep_filter]
                for h in range(0, y, self.size_stride_y):
                    for w in range(0, x, self.size_stride_x):
                        
                        
                        out_filter[x_step][y_step] = np.sum(image[w:w + self.width_filter, h:h + self.height_filter]*self.filters[:][:][num_filter][deep_filter])
                        
                        x_step = x_step + 1
                        if x_step == x_layer:
                            break
                    y_step = y_step + 1
                    x_step = 0
                    if y_step == y_layer:
                        break
                
                out_channel.append(out_filter.copy())
                
                #out_channel = []
            #print(out_channel)
            out_filter_result = np.zeros((x_layer,y_layer), float)
            for matrix in out_channel:
                out_filter_result = np.add(out_filter_result, matrix)
            out_filter_result = np.add(out_filter_result, self.bias[num_filter])
            out_image.append(out_filter_result.copy())
            #print()
        #print(out_image)
        return out_image
    def padding(self, in_image):
        return np.pad(in_image, pad_width=[[self.size_padding[0],self.size_padding[0]],[self.size_padding[1],self.size_padding[1]]], mode='constant', constant_values=0)

In [87]:
class MaxPool2d():
    def __init__(self, kernel_size, padding = [0,0], stride = [1,1]):
        self.size_padding = padding
        self.size_stride_x = stride[0]
        self.size_stride_y = stride[1]
        self.width_filter = kernel_size[0]
        self.height_filter = kernel_size[1]
    def padding(self, in_image):
        return np.pad(in_image, pad_width=[[self.size_padding[0],self.size_padding[0]],[self.size_padding[1],self.size_padding[1]]], mode='constant', constant_values=0)
    def forward(self, in_image):
        kanals, y, x = np.array(in_image).shape
        x_layer = int((x - self.width_filter + 2*self.size_padding[0])/self.size_stride_x + 1)
        y_layer = int((y - self.height_filter + 2*self.size_padding[0])/self.size_stride_y + 1)
        kanals, y, x = np.array(in_image).shape
        for i in range(kanals):
            in_image[i] = self.padding(in_image[i])
        #in_image = self.padding(in_image)
        kanals, y, x = np.array(in_image).shape
        out_channel = np.zeros((x_layer,y_layer), float)
        out_image = []
        for kanal in range(kanals):
            
            x_step = 0
            y_step = 0
            image = in_image[kanal]
            for h in range(0, y, self.size_stride_y):
                for w in range(0, x, self.size_stride_x):
                    out_channel[x_step][y_step] = max(map(max, image[w:w + self.width_filter, h:h + self.height_filter]))
                    x_step = x_step + 1
                    if x_step == x_layer:
                        break
                y_step = y_step + 1
                x_step = 0
                if y_step == y_layer:
                    break
            
            #out_channel = []
        #print(out_channel)
            out_image.append(out_channel.copy())
        #print()
    #print(out_image)
        return out_image
        

In [115]:
class Linear():
    def __init__(self, input_size, out_size, type_act):
        self.input_neurons = input_size
        self.out_neurons = out_size
        self.weights = [[random.random() for width in range(self.input_neurons)] for height in range(self.out_neurons)]
        self.bias = [1 for i in range(self.out_neurons)]
        self.act = type_act
    def forward(self, in_image):
        result = []
        for i in range(self.out_neurons):
            summ = 0
            for j in range(self.input_neurons):
                summ = summ + in_image[j]* self.weights[i][j]
            
            summ = summ + self.bias[i]
            neuron = 0
            if self.act = 'relu':
                neuron = self.relu(summ)
            else:
                neuron = self.softmax(summ)
            result.append(neuron)
        return result
    def relu(self, summ_b):
        return np.maximum(0, summ_b)
    def softmax(self, x):
        y = np.exp(x - np.max(x, axis=1, keepdims=True))
        return y / np.sum(y, axis=1, keepdims=True)

In [ ]:
class SoftmaxCrossEntropy():
    def loss(self, probs, label):
        loss = -np.sum(label * np.log(probs))
        return loss

    

In [114]:
arr = [1,5,5,5,1]
lin = Linear(5, 7)
arr1 = lin.forward(arr)
arr1

7.718741870813408
9.411072445442368
10.716143998136713
6.629588401299365
11.377516095355656
6.937899852880771
11.066586884627288


[7.718741870813408,
 9.411072445442368,
 10.716143998136713,
 6.629588401299365,
 11.377516095355656,
 6.937899852880771,
 11.066586884627288]

In [95]:
arr = [5,1,2,3, -1]
np.maximum(0,arr)

array([5, 1, 2, 3, 0])

In [82]:
image =[[[1,1,1,1,1],
        [1,1,1,1,1],
        [1,1,1,1,1],
        [1,1,1,1,1],
        [1,1,1,1,1]],
        [[1,1,1,1,1],
        [1,1,1,1,1],
        [1,1,1,1,1],
        [1,1,1,1,1],
        [1,1,1,1,1]],
        [[1,1,1,1,1],
        [1,1,1,1,1],
        [1,1,1,1,1],
        [1,1,1,1,1],
        [1,1,1,1,1]]]
conv = convolution(in_channels=3, out_channels=6, kernel_size=(5, 5), stride=(5, 5), padding = (5, 5))


In [83]:
asd = conv.forward(image)


In [90]:
i = [[1,2], [3,4]]
i[0][1]

2

In [77]:
np.array(asd).shape

(6, 5, 5)

In [84]:
asd

[array([[ 1.        ,  1.        ,  1.        ],
        [ 1.        , 39.69846019,  1.        ],
        [ 1.        ,  1.        ,  1.        ]]),
 array([[ 1.        ,  1.        ,  1.        ],
        [ 1.        , 33.76081278,  1.        ],
        [ 1.        ,  1.        ,  1.        ]]),
 array([[ 1.        ,  1.        ,  1.        ],
        [ 1.        , 38.46123376,  1.        ],
        [ 1.        ,  1.        ,  1.        ]]),
 array([[ 1.        ,  1.        ,  1.        ],
        [ 1.        , 38.45846036,  1.        ],
        [ 1.        ,  1.        ,  1.        ]]),
 array([[ 1.        ,  1.        ,  1.        ],
        [ 1.        , 38.34074474,  1.        ],
        [ 1.        ,  1.        ,  1.        ]]),
 array([[ 1.       ,  1.       ,  1.       ],
        [ 1.       , 39.4569844,  1.       ],
        [ 1.       ,  1.       ,  1.       ]])]

In [88]:
pool = MaxPool2d(kernel_size=(2, 2), stride=(1, 1))


In [89]:
pool.forward(asd)

[array([[39.69846019, 39.69846019],
        [39.69846019, 39.69846019]]),
 array([[33.76081278, 33.76081278],
        [33.76081278, 33.76081278]]),
 array([[38.46123376, 38.46123376],
        [38.46123376, 38.46123376]]),
 array([[38.45846036, 38.45846036],
        [38.45846036, 38.45846036]]),
 array([[38.34074474, 38.34074474],
        [38.34074474, 38.34074474]]),
 array([[39.4569844, 39.4569844],
        [39.4569844, 39.4569844]])]

In [ ]:
array = [[1,1],[1,1]]
np.pad(array, pad_width=[[0,0],[0,0]], mode='constant', constant_values=0)

In [ ]:
out_filter = np.zeros(3,3)